In [1]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [2]:
%load_ext autoreload
%autoreload 2
from bnpm import server

QObject::moveToThread: Current thread (0x1381e10) is not the object's thread (0x1d2ea80).
Cannot move to target thread (0x1381e10)

QObject::moveToThread: Current thread (0x1381e10) is not the object's thread (0x1d2ea80).
Cannot move to target thread (0x1381e10)

QObject::moveToThread: Current thread (0x1381e10) is not the object's thread (0x1d2ea80).
Cannot move to target thread (0x1381e10)

QObject::moveToThread: Current thread (0x1381e10) is not the object's thread (0x1d2ea80).
Cannot move to target thread (0x1381e10)

QObject::moveToThread: Current thread (0x1381e10) is not the object's thread (0x1d2ea80).
Cannot move to target thread (0x1381e10)

QObject::moveToThread: Current thread (0x1381e10) is not the object's thread (0x1d2ea80).
Cannot move to target thread (0x1381e10)

QObject::moveToThread: Current thread (0x1381e10) is not the object's thread (0x1d2ea80).
Cannot move to target thread (0x1381e10)

QObject::moveToThread: Current thread (0x1381e10) is not the object's thread

In [3]:
import os
from pathlib import Path
import functools
import tempfile
import getpass
import gc

In [4]:
import numpy as np
import natsort

------------
# ==== Start ====

## Prepare username + password

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = False

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username:  rh183
Password:  ········


## Prepare `sftp` and `ssh` objects

In [6]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)

In [7]:
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [8]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [9]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

In [10]:
prompt_snip = f'[{username}'

In [11]:
send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

# Batch run

In [12]:
mouse = 'mouse_0403R'

In [13]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path=f'/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/{mouse}/',
    search_pattern_re='exp',
    max_depth=5,
    find_folders=True,
    find_files=False,
    verbose=True,
))

/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403R/20230708/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403R/20230709/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403R/20230702/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403R/20230703/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403R/20230707/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403R/20230706/scanimage_data/exp


In [14]:
dates_toUse = natsort.natsorted([Path(p).parts[-3] for p in paths_found])
dates_toUse

['20230702', '20230703', '20230706', '20230707', '20230708', '20230709']

## Start a small interactive node

In [15]:
send_expect(f'srun --pty -p interactive -t 0-00:30:00 --mem=2G /bin/bash');

bash --pty -p interactive -t 0-00:30:00 --mem=2G /bin/ 

srun: job 31500428 queued and waiting for resources

srun: job 31500428 has been allocated resources

(base) [rh183@compute-a-16-163 ~]$ 


## Pull/update repo

In [16]:
send_expect('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP')
send_expect('git pull');

c
_repos/NBAPhms/neurobio/sabatini/rich/github 
(base) [rh183@compute-a-16-163 NBAP]$ 
g
it pull

Already up-to-date.

(base) [rh183@compute-a-16-163 NBAP]$ 


## Activate environment

In [17]:
name_env = '/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'

send_expect(f'conda activate {name_env}');

con
ini/rich/virtual_envs/FR/neurobio/sabat 

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-163 NBA


## Prepare commands

In [17]:
send_expect('pip install bnpm[core_cv2Headless] face_rhythm[all_cv2Headless] --upgrade')

pip 
install bnpm[core_cv2Headless] face_rhythm[all_cv2Headless] --upgrade



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.6/195.6 kB 6.4 MB/s eta 0:00:00













  Attempting uninstall: bnpm
    Found existing installation: bnpm 0.3.4
    Uninstalling bnpm-0.3.4:
      Successfully uninstalled bnpm-0.3.4


AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-164 NBA


('(/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-164 NBA\rAP]$ ',
 True)

In [18]:
for date in dates_toUse:
# for date in ['20230419']:

    # mouse = 'mouse_0916N'
    # date = '20230501'

    name_slurm = mouse[-3:] + '_' + date[-4:]

    dir_s2p       = str(Path('/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output') / mouse / date / 'scanimage_data' / 'exp' / 'jobNum_0' / 'suite2p' / 'plane0' )

    dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm') / mouse / date )


    name_job = 'jobNum_'


    path_script_remote      = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py'))  ## path to a copy of the script (.py) file to run within the repo on the server

    path_dispatcher_local   = str(Path(r'/media/rich/Home_Linux_partition/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/dispatcher.py'))  ## path to the dispatcher.py file on local computer
    path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server

    ## Expectation for dispatch args: 
    # path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env


    commands = {
    #     'make_dir': f"mkdir -p {dir_data_remote}",
    #     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
    #     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
    'dispatch': f"\
    python3 \
    {path_dispatcher_remote} \
    {path_script_remote} \
    {dir_saveOutputs} \
    {dir_s2p} \
    {name_job} \
    {name_slurm} \
    {name_env}",
    }
    display(commands)
    
    sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
    sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));
    
    send_expect(commands['dispatch']);

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230702/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230702     /n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403R/20230702/scanimage_data/exp/jobNum_0/suite2p/plane0     jobNum_     03R_0702     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

tqm/mouse_0403R/20230702/dispatcher.py abatini/rich/analysis/make_dFoF_and_ 
0702     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    jobNum_     03R_ 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR


Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py to /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230702/make_dFoF_and_tqm.py

Submitting job: jobNum_ 0

Submitted batch job 31500433

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-163 NBA


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230703/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230703     /n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403R/20230703/scanimage_data/exp/jobNum_0/suite2p/plane0     jobNum_     03R_0703     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

 
0703     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    jobNum_     03R_ 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py to /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230703/make_dFoF_and_tqm.py

Submitting job: jobNum_ 0
Submitted batch job 31500435

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-163 NBA


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230706/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230706     /n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403R/20230706/scanimage_data/exp/jobNum_0/suite2p/plane0     jobNum_     03R_0706     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

...

    p
0707     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    jobNum_     03R_ 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py to /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230707/make_dFoF_and_tqm.py

Submitting job: jobNum_ 0

Submitted batch job 31500438

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-163 NBA


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230708/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230708     /n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403R/20230708/scanimage_data/exp/jobNum_0/suite2p/plane0     jobNum_     03R_0708     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

   
0708     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    jobNum_     03R_ 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py to /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230708/make_dFoF_and_tqm.py

Submitting job: jobNum_ 0
Submitted batch job 31500440

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-163 NBA


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230709/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230709     /n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403R/20230709/scanimage_data/exp/jobNum_0/suite2p/plane0     jobNum_     03R_0709     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

 
0709     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    jobNum_     03R_ 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py to /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230709/make_dFoF_and_tqm.py

Submitting job: jobNum_ 0
Submitted batch job 31500441

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-163 NBA


### check on job

### Check if it worked

In [19]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path='/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N',
    search_pattern_re='dFoF.npy',
    max_depth=9,
    find_folders=False,
    find_files=True,
))
paths_found

/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231205/jobNum_0/dFoF.npy
/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231029/jobNum_0/dFoF.npy
/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231127/jobNum_0/dFoF.npy
/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231024/jobNum_0/dFoF.npy
/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231021/jobNum_0/dFoF.npy
/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231025/jobNum_0/dFoF.npy
/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231106/jobNum_0/dFoF.npy
/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231129/jobNum_0/dFoF.npy
/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231023/jobNum_0/dFoF.npy
/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/2023102

['/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231021/jobNum_0/dFoF.npy',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231022/jobNum_0/dFoF.npy',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231023/jobNum_0/dFoF.npy',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231024/jobNum_0/dFoF.npy',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231025/jobNum_0/dFoF.npy',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231026/jobNum_0/dFoF.npy',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231027/jobNum_0/dFoF.npy',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231028/jobNum_0/dFoF.npy',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231029/jobNum_0/dFoF.npy',
 '/n/data1/hms/neurobio/sabatini/rich/analysis

### Download results

In [20]:
dates = [Path(p).parts[-4] for p in paths_found]

In [21]:
paths_download = {d: p for d,p in zip(dates, paths_found)}

In [24]:
sftp.get_dir(
    source=r'/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/',
    target=r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0916N/dFoF_and_tqm/',
)

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231205/parameters_batch.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0916N/dFoF_and_tqm/20231205/parameters_batch.json


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231205/jobNum_0/tqm.pkl   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0916N/dFoF_and_tqm/20231205/jobNum_0/tqm.pkl


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231205/jobNum_0/params.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0916N/dFoF_and_tqm/20231205/jobNum_0/params.json


...

0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231101/dispatcher.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0916N/dFoF_and_tqm/20231101/dispatcher.py


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0916N/20231101/notes.txt   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0916N/dFoF_and_tqm/20231101/notes.txt


0.00b [00:00, ?b/s]


## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()